<a href="https://colab.research.google.com/github/opread/Coursera-AI-Agents-do/blob/main/copie_a_blocnotesului_programmaticprompting1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
from __future__ import annotations

import re
from pathlib import Path
from typing import Dict, List, Tuple

# If you already have your own `completion(...)` wrapper, keep it.
# Otherwise, uncomment and adapt the OpenAI client example below.
#
# from openai import OpenAI
# client = OpenAI()
#
# def completion(*, model: str, messages: List[Dict[str, str]], max_tokens: int):
#     return client.chat.completions.create(
#         model=model,
#         messages=messages,
#         max_tokens=max_tokens
#     )


def generate_response(messages: List[Dict[str, str]]) -> str:
    """Call LLM to get response."""
    response = completion(
        model="openai/gpt-4",
        messages=messages,
        max_tokens=1024,
    )
    # OpenAI chat completion returns role + content; we only need the text here.
    return response.choices[0].message.content or ""


_CODE_FENCE_RE = re.compile(
    r"```(?:\s*python)?\s*\n(.*?)\n```",
    re.DOTALL | re.IGNORECASE,
)


def extract_code_block(response: str) -> str:
    """
    Extract the first fenced code block from `response`.

    Accepts:
      ```python
      <code>
      ```
      or
      ```
      <code>
      ```

    If no fenced block is found, returns the original response.
    """
    match = _CODE_FENCE_RE.search(response)
    if match:
        return match.group(1).strip()
    return response.strip()


def develop_custom_function() -> Tuple[str, str, str]:
    """Interactive flow: generate function, then docs, then tests."""
    print("\nWhat kind of function would you like to create?")
    print("Example: 'A function that calculates the factorial of a number'")
    function_description = input("Your description: ").strip()

    # Initialize conversation with system prompt
    messages: List[Dict[str, str]] = [
        {"role": "system", "content": "You are a Python expert helping to develop a function."}
    ]

    # First prompt — Basic function
    messages.append({
        "role": "user",
        "content": (
            "Write a Python function that "
            f"{function_description}. Output the function in a ```python code block```."
        ),
    })
    initial_function_resp = generate_response(messages)
    initial_function = extract_code_block(initial_function_resp)

    print("\n=== Initial Function ===")
    print(initial_function)

    # Add assistant's (code-only) response to conversation
    messages.append({
        "role": "assistant",
        "content": f"```python\n{initial_function}\n```",
    })

    # Second prompt — Add documentation
    messages.append({
        "role": "user",
        "content": (
            "Add comprehensive documentation to this function, including description, parameters, "
            "return value, examples, and edge cases. Output the function in a ```python code block```."
        ),
    })
    documented_function_resp = generate_response(messages)
    documented_function = extract_code_block(documented_function_resp)

    print("\n=== Documented Function ===")
    print(documented_function)

    # Add documentation response (code-only) to conversation
    messages.append({
        "role": "assistant",
        "content": f"```python\n{documented_function}\n```",
    })

    # Third prompt — Add unittest test cases
    messages.append({
        "role": "user",
        "content": (
            "Add unittest test cases for this function, including tests for basic functionality, "
            "edge cases, error cases, and various input scenarios. Output the code in a ```python code block```."
        ),
    })
    test_cases_resp = generate_response(messages)
    test_cases = extract_code_block(test_cases_resp)

    print("\n=== Test Cases ===")
    print(test_cases)

    # Generate a safe filename from the description
    safe_stem = "".join(ch.lower() if ch.isalnum() or ch.isspace() else " " for ch in function_description)
    safe_stem = "_".join(safe_stem.split())[:30] or "function"
    filename = f"{safe_stem}.py"

    # Save final version
    out_path = Path(filename)
    out_path.write_text(f"{documented_function}\n\n{test_cases}\n", encoding="utf-8")

    return documented_function, test_cases, filename


if __name__ == "__main__":
    func_code, tests_code, out_file = develop_custom_function()
    print(f"\nFinal code has been saved to {out_file}")



What kind of function would you like to create?
Example: 'A function that calculates the factorial of a number'
Your description: calculeaza 1:n factorial

=== Initial Function ===
def calculate_factorials(n):
    factorials = []
    fact = 1
    for i in range(1, n+1):
        fact *= i
        factorials.append(fact)
    return factorials

n = 5
print(calculate_factorials(n))

=== Documented Function ===
def calculate_factorials(n):
    """
    Function to calculate the factorial of numbers from 1 to n. 
    
    The factorial of a non-negative integer n is the product of all positive integers less than or equal to n. 
    It is denoted by n!, and defined as n! = n * (n-1) * (n-2) * ... * 3 * 2 * 1.
    
    Parameters:
    n (int): The upper limit till where the factorials has to be calculated. It is a non-negative integer.
    
    Returns:
    list: A list of integers representing the factorials of numbers from 1 to n.
    
    Example:
    >>> calculate_factorials(5)
    [1, 2, 